In [17]:
# import libraries
from datasets import load_dataset
from textblob import TextBlob
import os

import torch

from gensim.models.keyedvectors import load_word2vec_format

In [18]:
# download dataset
ds = load_dataset("tner/ontonotes5")

KeyboardInterrupt: 

In [ ]:
entity2tag = {
    "O": 0,
    "B-CARDINAL": 1,
    "B-DATE": 2,
    "I-DATE": 3,
    "B-PERSON": 4,
    "I-PERSON": 5,
    "B-NORP": 6,
    "B-GPE": 7,
    "I-GPE": 8,
    "B-LAW": 9,
    "I-LAW": 10,
    "B-ORG": 11,
    "I-ORG": 12, 
    "B-PERCENT": 13,
    "I-PERCENT": 14, 
    "B-ORDINAL": 15, 
    "B-MONEY": 16, 
    "I-MONEY": 17, 
    "B-WORK_OF_ART": 18, 
    "I-WORK_OF_ART": 19, 
    "B-FAC": 20, 
    "B-TIME": 21, 
    "I-CARDINAL": 22, 
    "B-LOC": 23, 
    "B-QUANTITY": 24, 
    "I-QUANTITY": 25, 
    "I-NORP": 26, 
    "I-LOC": 27, 
    "B-PRODUCT": 28, 
    "I-TIME": 29, 
    "B-EVENT": 30,
    "I-EVENT": 31,
    "I-FAC": 32,
    "B-LANGUAGE": 33,
    "I-PRODUCT": 34,
    "I-ORDINAL": 35,
    "I-LANGUAGE": 36
}


In [ ]:
df_train = ds["train"].to_pandas()
df_val = ds["validation"].to_pandas()
df_test = ds["test"].to_pandas()

df_train.head()

,tokens,tags
0,"[People, start, their, own, businesses, for, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[But, a, chance, to, fill, out, sales, -, tax,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,"[Red, tape, is, the, bugaboo, of, small, busin...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[Ironically, ,, the, person, who, wants, to, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Yet, every, business, owner, has, to, face, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def analyze_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity < 0:
        return 0  # Negativo
    elif polarity == 0:
        return 1  # Neutro
    else:
        return 2  # Positivo
        
def process_sentences(df):
    # Crear la columna 'sentence' uniendo los tokens
    df["sentence"] = df["tokens"].apply(lambda x: " ".join(x))

    # Aplicar análisis de sentimiento
    df["SA"] = df["sentence"].apply(analyze_sentiment)
    
    return df

# Aplicar a cada dataset
df_train = process_sentences(df_train)
df_val = process_sentences(df_val)
df_test = process_sentences(df_test)

In [ ]:
df_train.head()

,tokens,tags,sentence,SA
0,"[People, start, their, own, businesses, for, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",People start their own businesses for many rea...,2
1,"[But, a, chance, to, fill, out, sales, -, tax,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",But a chance to fill out sales - tax records i...,2
2,"[Red, tape, is, the, bugaboo, of, small, busin...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",Red tape is the bugaboo of small business .,0
3,"[Ironically, ,, the, person, who, wants, to, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Ironically , the person who wants to run his o...",2
4,"[Yet, every, business, owner, has, to, face, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Yet every business owner has to face the mound...,2


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# EJEMPLO DE INTERNET

#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    print(ranking[i])
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


0
1) negative 0.7236
1
2) neutral 0.2287
2
3) positive 0.0477


In [19]:
def obtain_sa_roberta(sentence: str):
    sentence = preprocess(sentence)
    encoded_input = tokenizer(sentence, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    return ranking[0]

labeled_correctly = 0
for df in [df_train]:  # [df_train, df_val, df_test]
    df["SA_Roberta"] = df["sentence"].apply(obtain_sa_roberta)

labeled_correctly += (df["SA"] == df["SA_Roberta"]).sum()
labeled_correctly

31010

In [23]:
df["SA"].tail()

59919    1
59920    1
59921    1
59922    1
59923    1
Name: SA, dtype: int64

In [24]:
df["SA_Roberta"].tail()

59919    1
59920    1
59921    1
59922    2
59923    1
Name: SA_Roberta, dtype: int64

In [22]:
df_train.to_csv(f"data/train_roberta.csv", index=False)

In [ ]:
# save data in .csv
if not os.path.exists("data"):
    os.makedirs("data")

df_train.to_csv(f"data/train.csv", index=False)
df_val.to_csv(f"data/val.csv", index=False)
df_test.to_csv(f"data/test.csv", index=False)

In [ ]:
# Load embeddings model to use in the assignment
w2v_model = load_word2vec_format("embeddings/GoogleNews-vectors-negative300.bin.gz", binary = True)